In [1]:
from pyconsolida.budget_reader import *
from pyconsolida.budget_reader import _get_valid_costo_rows, _read_raw_budget_sheet
from pathlib import Path
import pandas as pd
import flammkuchen as fl
import numpy as np

FILENAME =  "/Users/vigji/Desktop/icop/2021/21_12 Dicembre/9997/Analisi.xls"

FASE = "Spese Generali"

In [ ]:
raw_df = pd.read_excel(Path(FILENAME), sheet_name=None)
for fase, df_fase in list(raw_df.items()):
    if fase == FASE:
        print(fase)
        # costi_fase = _read_raw_budget_sheet(df_fase)
        df = translate_df(df_fase)
        # Trova l'inizio delle righe costi, e return se non ce ne sono:
        df_costi = crop_costi(df)

        # Aggiungi colonna con tipologie, trascinando in basso ogni nuovo header tipologia:
        df_costi = add_tipologia_column(df_costi)

        # Seleziona righe con un codice costo valido:
        selection = _get_valid_costo_rows(df_costi)

        voci_costo = df_costi.iloc[selection, :].copy()
        fix_types(voci_costo)
        voci_costo = voci_costo[voci_costo[HEADERS["quantita"]] > 0]

## Compute difference

In [74]:
data_dict = fl.load("/Users/vigji/Desktop/icop/exported_luigi/230314_115521_python_data.h5")

budgets_tot, budgets_dec = data_dict["budgets"], data_dict["budgets_dec"]

budgets_tot = budgets_tot.set_index(["commessa", "codice"]).drop(["costo u.", "imp.comp."], axis=1)
budgets_dec = budgets_dec.set_index(["commessa", "codice"]).drop(["costo u.", "imp.comp."], axis=1)

dec_al, tot_al = budgets_dec.align(budgets_tot)
dec_al.loc[:, numerical] = dec_al.loc[:, numerical].fillna(0)
tot_al.loc[:, numerical] = tot_al.loc[:, numerical].fillna(0)

print("Before dropping weird negatives: ", (tot_al["quantita"] - dec_al["quantita"]).sum())

problematic = dec_al[(dec_al["quantita"] > tot_al["quantita"]) & (tot_al["quantita"] > 0)].index

dec_al = dec_al.drop(problematic)
tot_al = tot_al.drop(problematic)

print("before dropping all negatives: ", (tot_al["quantita"] - dec_al["quantita"]).sum())
negative_correct = dec_al[(dec_al["quantita"] > tot_al["quantita"])].index

dec_al = dec_al.drop(negative_correct)
tot_al = tot_al.drop(negative_correct)

print("dropping all negatives: ", (tot_al["quantita"] - dec_al["quantita"]).sum())

Before dropping weird negatives:  9115614.854898594
before dropping all negatives:  9158804.924472522
dropping all negatives:  9183882.986377284


9158804.924472522

In [34]:
dec_al, tot_al = budgets_dec.align(budgets_tot)
all_diffs = dec_al["quantita"] - tot_al["quantita"]

not_matching_2021 = set(budgets_dec.index) - set(budgets_tot.index)
negative_diffs = set(all_diffs[all_diffs < 0].index)
budgets_dec.drop(list(negative_diffs - not_matching_2021))

dec_al, tot_al = budgets_dec.align(budgets_tot)
numerical = ["quantita"]
dec_al.loc[:, numerical] = dec_al.loc[:, numerical].fillna(0)
tot_al.loc[:, numerical] = tot_al.loc[:, numerical].fillna(0)

not_matching_2021 = set(budgets_dec.index) - set(budgets_tot.index)
negative_diffs = set(all_diffs[all_diffs < 0].index)

tot_al = tot_al.drop(list(negative_diffs))
dec_al = dec_al.drop(list(negative_diffs))

In [39]:
(tot_al["quantita"] - dec_al["quantita"]).sum()

7320548.651928511

In [25]:
(tot_al["quantita"] - dec_al["quantita"]).sum()

9115614.854898594

In [158]:
all_diffs = dec_al - 

In [159]:
s = budgets_dec.loc[list(negative_diffs - not_matching_2021)]

1223

In [173]:
print("December 2021")
budgets_dec.drop(list(negative_diffs - not_matching_2021))

December 2021


tipologia  \
commessa codice                            
1432     6                  Noli interni   
         17                 Noli interni   
         87                 Noli interni   
         89                 Noli interni   
         145                Noli interni   
...                                  ...   
1449     919     Spese varie di gestione   
         920     Spese varie di gestione   
         933     Spese varie di gestione   
         934     Spese varie di gestione   
         943     Spese varie di gestione   

                                                  voce     u.m.      quantita  \
commessa codice                                                                 
1432     6           Agitatore Lorenzetto L10000 (ICP)        g     16.000000   
         17                         Autovettura (ICOP)        g    102.000000   
         87                   Camion Gru Scania (ICOP)        h     45.000000   
         89                       Camion Scania (ICOP)        h     22.000000   
         145           Cisterna Gasolio 3000 L (digas)        g     84.000000   
...                                                ...      ...           ...   
1449     919       Charges Mandataire + Gerant (0,5% )  Forfait    740.000000   
         920                       Overhead ICOP (10%)      cad  14800.000000   
         933                                 Taxe CVAE       LS      0.002894   
         934                          Taxe CSS organic       LS      0.002631   
         943     Provvisions penalitées et reparations  Forfait      0.500000   

                 costo u.  imp.comp.  
commessa codice                       
1432     6           20.0      320.0  
         17          40.0     4080.0  
         87          60.0     2700.0  
         89          45.0      990.0  
         145          1.0       84.0  
...                   ...        ...  
1449     919          1.0        0.0  
         920          1.0        0.0  
         933     225000.0        0.0  
         934      90000.0        0.0  
         943      25000.0        0.0  

[3522 rows x 6 columns]

In [165]:
print("Sum 2022")
budgets_tot.loc[list(negative_diffs - not_matching_2021)].head()

Sum 2022


,,tipologia,voce,u.m.,quantita,costo u.,imp.comp.
commessa,codice,,,,,,
1408,240,Materiali d'opera,Materiale per preparazione testa palo,corpo,0.500000,1200.0,600.00000
1437,1118,Materiali di consumo,Legname,cad,6.686000,500.0,3343.00000
5014,900,Materiali d'opera,Pietre per sottofondo,ton,498.400000,15.0,7476.00000
1437,1301,Noli esterni - prestazioni,Trasporto con ribassato (LISCIO),h,23.117588,85.0,1964.99498
5014,903,Materiali d'opera,Misto di fiume Se.Cla,ton,2887.000000,13.5,38974.50000


In [112]:
all_diffs = tot_al[numerical] - dec_al[numerical]

In [117]:
all_diffs["quantita"].sum()

9115614.854898594

In [ ]:
 17930866.01